In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Ella Fitzgerald,Darktown Strutters' Ball,I'll be down to get you in a taxi honey \r\nB...
1,Bread,Didn't Even Know Her Name,Should've kept on goin' \r\nBut the hurt was ...
2,Jimmy Buffett,Makin' Music For Money,When I woke up this morning \r\nI was tired a...
3,Dusty Springfield,All I Have To Offer You Is Love,"If all you want's a good time, baby \r\nThere..."
4,Frankie Goes To Hollywood,Two Tribes,The air attack warning sounds like. \r\nThis ...
5,Helloween,I Can,May I take you higher \r\nFeed well your desi...
6,Marilyn Manson,They Said That Hell's Not Hot,I kill myself in small amounts \r\nIn each re...
7,Cyndi Lauper,Yeah Yeah,Through the burning sun \r\nAnd the driving r...
8,Idina Menzel,Once Upon A Time,All the king's horses and all the king's men C...
9,Roxette,Lies,Put 'em on \r\nPut the big boots on \r\nBaby...


In [9]:
df['text'][0]

"I'll be down to get you in a taxi honey  \r\nBetter be ready 'bout half past eight  \r\nI mean don't be late  \r\nBe there when the band starts playin'  \r\nRemember when you get there honey  \r\nDance all over the floor  \r\nDance all over my shoes  \r\nWhen the band plays the Jelly Roll blues  \r\nSet 'em all alight  \r\n  \r\nAt the darktown strutters ball  \r\nI'll be down to get you in a push cart honey  \r\nBetter be ready 'bout half past eight  \r\nI mean don't be late  \r\nBe there when the band starts playin'  \r\n  \r\n'Member when you get there honey  \r\nShim sham wins them all  \r\nDance all over my shoes  \r\nWhen the band plays the Jelly Roll's blues  \r\nSet 'em all alight  \r\n(At) the darktown strutters ball\r\n\r\n"

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [22]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [25]:
similarity[0]

array([1.        , 0.01193459, 0.04190794, ..., 0.02551756, 0.        ,
       0.        ])

In [26]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [29]:
def recommendation(song_title):
    # Check if the song exists in the DataFrame
    if not df[df['song'] == song_title].empty:
        idx = df[df['song'] == song_title].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

        songs = []
        for i in distances[1:6]:  # Assuming you want top 5 recommendations
            songs.append(df.iloc[i[0]].song)
        
        return songs
    else:
        return f"Song '{song_title}' not found in the dataset."

# Example usage:
result = recommendation('Crying Over You')
print(result)

Song 'Crying Over You' not found in the dataset.


In [30]:
recommendation('Crying Over You')

"Song 'Crying Over You' not found in the dataset."

In [31]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))